# Mock Data for Compressed Matrices

In [1]:
import numpy as np
import constriction
from tqdm import tqdm
import pickle

## Preparation

### Create Random Quantized Matrices

In [22]:
np.random.seed(202404151)
matrices = np.random.randn(100, 1024, 1024)

In [23]:
quantized_matrices = np.round(matrices * 8).astype(np.int8)
print(f'mins: {quantized_matrices.min(axis=(1, 2))}')
print(f'maxs: {quantized_matrices.max(axis=(1, 2))}')

mins: [-41 -38 -40 -38 -44 -38 -37 -41 -41 -37 -39 -40 -39 -37 -39 -40 -37 -41
 -40 -40 -39 -41 -38 -42 -39 -39 -42 -39 -41 -41 -38 -39 -40 -38 -38 -38
 -42 -41 -37 -44 -38 -38 -37 -39 -39 -36 -39 -42 -40 -40 -41 -38 -36 -41
 -40 -41 -42 -38 -38 -38 -42 -37 -39 -39 -38 -36 -39 -37 -38 -40 -43 -36
 -37 -37 -41 -39 -41 -37 -40 -42 -38 -37 -38 -37 -38 -42 -41 -42 -38 -42
 -40 -46 -39 -37 -42 -41 -37 -40 -38 -42]
maxs: [38 39 36 36 39 37 36 38 36 39 38 37 39 40 37 40 37 38 39 41 38 40 37 38
 36 36 39 39 36 39 37 37 37 38 39 39 41 41 37 36 37 38 35 37 37 40 42 36
 41 37 37 41 40 39 38 40 42 43 38 41 37 36 41 37 37 38 38 39 40 36 38 39
 39 40 39 38 39 37 38 40 39 43 43 44 36 42 44 39 43 39 37 37 38 43 40 38
 40 44 39 39]


### Create Entropy Models

In [4]:
values = []
counts = []
counts_12bit = []
entropies = []
cross_entropies_12bit = []
for quantized_matrix in tqdm(quantized_matrices):
    v, c = np.unique(quantized_matrix, return_counts=True)
    order = np.argsort(c)[::-1]
    v = v[order]
    c = c[order]
    values.append(v)
    counts.append(c)
    entropies.append(np.log2(quantized_matrix.size) - c @ np.log2(c) / quantized_matrix.size)
    c12bit = np.maximum(np.round(c / 256).astype(np.int32), 1)
    excess = sum(c12bit) - (1 << 12)
    assert excess >= 0 and excess <= len(c)
    if excess != 0:
        assert c12bit[excess - 1] > 1
    c12bit[:excess] -= 1
    counts_12bit.append(c12bit)
    cross_entropies_12bit.append(12 - c12bit @ np.log2(c12bit) / (1 << 12))
entropies = np.array(entropies)
cross_entropies_12bit = np.array(cross_entropies_12bit)
overheads = cross_entropies_12bit - entropies
print(f'Maximum absolute overhead: {overheads.max():.4f}')
print(f'Maximum relative overhead: {(overheads * 100 / entropies).max():.2f} %')
entropies, cross_entropies_12bit

100%|██████████| 100/100 [00:15<00:00,  6.47it/s]

Maximum absolute overhead: 0.0471
Maximum relative overhead: 0.93 %


(array([5.04873129, 5.04657062, 5.04677307, 5.04884595, 5.0473163 ,
        5.0486622 , 5.04661374, 5.04849787, 5.04695842, 5.04865761,
        5.04702978, 5.04732894, 5.04798469, 5.04782882, 5.04794853,
        5.04891135, 5.04778863, 5.0474892 , 5.04845118, 5.04870344,
        5.04692647, 5.0466256 , 5.04761184, 5.04772784, 5.04902663,
        5.04909297, 5.0483021 , 5.0477679 , 5.04680325, 5.0470743 ,
        5.04774866, 5.04810709, 5.04799223, 5.04751116, 5.04805406,
        5.04894296, 5.04914948, 5.04813692, 5.04773768, 5.04766498,
        5.04820142, 5.04674641, 5.04612653, 5.04954575, 5.04798425,
        5.04911074, 5.04806961, 5.04883531, 5.04730741, 5.04820349,
        5.0475805 , 5.04728574, 5.0476444 , 5.04874619, 5.04859174,
        5.04892564, 5.04605025, 5.04941888, 5.04747965, 5.04812066,
        5.04752634, 5.04696448, 5.04841835, 5.0466188 , 5.04660691,
        5.04749418, 5.04658043, 5.04970472, 5.04751697, 5.0466669 ,
        5.0487524 , 5.04709681, 5.0476797 , 5.04

In [5]:
inv_vocabs = [{v: i for i, v in enumerate(vs)} for vs in values]

### Entropy Coder

In [6]:
class AnsCoder:
    def __init__(self, precision, word_size, compressed=[]):
        self.precision = precision
        self.word_size = word_size
        self.word_mask = (1 << word_size) - 1
        self.quantile_mask = (1 << precision) - 1
        self.bulk = compressed.copy()
        self.head = 0
        while len(self.bulk) != 0 and (self.head >> word_size) == 0:
            self.head = (self.head << word_size) | self.bulk.pop()

    def push(self, symbol, m):
        if (self.head >> (2 * self.word_size - self.precision)) >= m[symbol]:
            self.bulk.append(self.head & self.word_mask)
            self.head >>= self.word_size

        z = self.head % m[symbol] + sum(m[0:symbol])
        self.head //= m[symbol]
        self.head = (self.head << self.precision) | z

    def pop(self, m):
        z = self.head & self.quantile_mask
        self.head >>= self.precision
        for symbol, m_symbol in enumerate(m):
            if z >= m_symbol:
                z -= m_symbol
            else:
                break
        self.head = self.head * m_symbol + z
        if (self.head >> self.word_size) == 0 and len(self.bulk) != 0:
            self.head = (self.head << self.word_size) | self.bulk.pop()
        return symbol

    def get_compressed(self):
        compressed = self.bulk.copy()
        head = self.head
        while head != 0:
            compressed.append(head & self.word_mask)
            head >>= self.word_size
        return compressed

## Compression 1: independently compressed matrix entries, with non-interleaved matrices

### Compress matrices

In [7]:
# with open("100-compressed-matrices.pickle", "rb") as f:
#     d = pickle.load(f)
#     compressed_stream = d["compressed_stream"]
#     coder_offsets = d["coder_offsets"]
#     del d

In [8]:
compressed_stream = []
coder_offsets = np.zeros((len(matrices), matrices[0].size), dtype=np.uint32)
coders = [AnsCoder(12, 16, [1, 1]) for _ in range(matrices[0].size)]
for i in tqdm(reversed(range(len(matrices)))):
    model = counts_12bit[i]
    inv_vocab = inv_vocabs[i]
    for j, entry, coder in zip(range(quantized_matrices[i].size - 1, -1, -1), reversed(quantized_matrices[i].ravel()), coders):
        symbol_id = inv_vocab[entry]
        coder.push(symbol_id, model)
        if len(coder.bulk) == 1:
            compressed_stream.append(coder.bulk[0])
            coder.bulk = []
        coder_offsets[i, j] = len(compressed_stream)
    print(f'Encoded matrix {i}; words emitted so far: {coder_offsets[i, -1]}')

1it [00:10, 10.33s/it]

Encoded matrix 99; words emitted so far: 0


2it [00:16,  7.64s/it]

Encoded matrix 98; words emitted so far: 0


3it [00:21,  6.83s/it]

...

Encoded matrix 2; words emitted so far: 31600869


99it [10:01,  6.02s/it]

Encoded matrix 1; words emitted so far: 31931007


100it [10:07,  6.07s/it]

Encoded matrix 0; words emitted so far: 32261990


In [9]:
for coder in coders:
    assert len(coder.bulk) == 0
    compressed = coder.get_compressed()
    assert len(compressed) == 2
    compressed_stream += compressed
compressed_stream = np.array(compressed_stream, dtype=np.uint16)

In [12]:
bitrate = len(compressed_stream) * 16 / quantized_matrices.size
len(compressed_stream), bitrate, bitrate - cross_entropies_12bit.mean()

(34691303, 5.293472747802735, 0.20617554845228003)

In [13]:
# with open("100-compressed-matrices.pickle", "wb") as f:
#     pickle.dump({"compressed_stream": compressed_stream, "coder_offsets": coder_offsets}, f)

In [6]:
with open("100-compressed-matrices.pickle", "rb") as f:
    d = pickle.load(f)
    compressed_stream = d["compressed_stream"]
    coder_offsets = d["coder_offsets"]
    del d

### Serialize to a File

In [34]:
def serialize_matrix(i):
    cdf = np.array([counts_12bit[i][:j].sum() for j in range(len(counts_12bit[i]) + 1)], dtype=np.uint16)
    data_offset = np.array([len(cdf) + 1], dtype=np.uint16)
    start = coder_offsets[i, 0]
    end = start if i == 0 else coder_offsets[i - 1, 0]
    data = compressed_stream[start:end][::-1]
    return np.concatenate([data_offset, cdf, data])

In [35]:
serialized_sizes = np.array([serialize_matrix(i).size for i in range(len(quantized_matrices))], dtype=np.uint32)
serialized_sizes, serialized_sizes.max()

(array([    78, 332241, 331060, 330216, 331987, 331455, 330044, 331860,
        332325, 329815, 331617, 332295, 329927, 331284, 332145, 331064,
        330066, 332319, 331558, 329879, 332840, 332339, 328111, 332194,
        333385, 328425, 330857, 335742, 327901, 329529, 336072, 328910,
        327637, 336437, 330944, 326216, 336044, 333434, 323793, 334167,
        337582, 322769, 331419, 340793, 323443, 326455, 343883, 326798,
        320542, 345330, 331853, 314933, 343560, 340227, 309240, 339272,
        350052, 307086, 329680, 360216, 309468, 315432, 368840, 319216,
        297229, 373021, 335884, 279694, 367900, 360475, 265178, 351095,
        390701, 260596, 321099, 419455, 272920, 276076, 444041, 304387,
        219989, 457270, 353557, 166826, 444379, 419030, 140050, 383622,
        503638, 152196, 253879, 616465, 180265,  78305, 761215, 213233,
         12997, 792229, 250265,   6522], dtype=uint32),
 792229)

In [81]:
# with open("100-compressed-matrices.bin", "wb") as f:
#     initial_coder_heads = compressed_stream[coder_offsets[0, 0]:][::-1]
#     initial_coder_heads.tofile(f)
#     serialized_sizes_lo = (serialized_sizes & ((1 << 16) - 1)).astype(np.uint16)
#     serialized_sizes_hi = (serialized_sizes >> 16).astype(np.uint16)
#     serialized_sizes_le = np.vstack([serialized_sizes_lo, serialized_sizes_hi]).T.ravel()
#     serialized_sizes_le.tofile(f)
#     for i in range(len(quantized_matrices)):
#         m = serialize_matrix(i)
#         m.tofile(f)
#     # NOTE: this doesn't serialize the final compressed data that would be necessary
#     # to reconstruct the original coder states (which is currently always `[1, 0]`).

In [7]:
!ls -lh 100-compressed-matrices.bin

-rw-rw-r-- 1 robamler robamler 67M Mai 12 16:47 100-compressed-matrices.bin


## Debugging

In [24]:
# Checksum adapted from rust's FxHash, but restricted to 26 bit hashes so that the
# multiplication doesn't exceed the range of exactly representable integers in JavaScript.
def get_checksum(i):
    checksum = 0
    for value in quantized_matrices[i].ravel():
        value = inv_vocabs[i][value]
        checksum = ((checksum & 0x001f_ffff) << 5) | (checksum >> 21) # rotate
        checksum = (checksum ^ value) * 0x0322_0a95
        checksum = checksum & 0x03ff_ffff # truncate to 26 bit
    return checksum

In [57]:
[inv_vocabs[1][i] for i in quantized_matrices[1].ravel()[[32_768]]]

[13]

In [ ]:
for i in range(100):
    print(f'checksum of matrix {i}: {get_checksum(i)}')

checksum of matrix 0: 55445785
checksum of matrix 1: 15163511
checksum of matrix 2: 6664559
checksum of matrix 3: 15747500
checksum of matrix 4: 5573655
checksum of matrix 5: 27711411
checksum of matrix 6: 20047205
checksum of matrix 7: 10641525
checksum of matrix 8: 64606195
checksum of matrix 9: 65232545
checksum of matrix 10: 51903642
checksum of matrix 11: 57475160
checksum of matrix 12: 54761909
checksum of matrix 13: 30428408
checksum of matrix 14: 22313942
checksum of matrix 15: 33179881
checksum of matrix 16: 37189258
checksum of matrix 17: 16278593
checksum of matrix 18: 56573661
checksum of matrix 19: 41121083
checksum of matrix 20: 55929650
checksum of matrix 21: 43756132
checksum of matrix 22: 28003007
checksum of matrix 23: 56424187
checksum of matrix 24: 28589585
checksum of matrix 25: 37515944
checksum of matrix 26: 58783295
checksum of matrix 27: 57634150
checksum of matrix 28: 6008159
checksum of matrix 29: 16510335
checksum of matrix 30: 64138918
checksum of matrix 31